# From RAGs to riches: Build an AI document interrogation app in 30 mins

Dharhas Pothina | PyData NYC 2023

---

## Hi! I'm Dharhas 👋

CTO at Quansight

<!-- ![caption](images/viking_boat_desk.jpg) -->


<img src="images/viking_boat_desk.jpg" width=50%/>

## Today's goal: Build this!

**TODO (opitonal): Screenshot of the final app for initial motivation**

## Retrieval-Augmented Generation (RAG): Make LLMs more useful

LLMs are trained on vast datasets, which has a drawback: hallucination.

<img src="images/chatgpt_what_is_ragna.png"/>

RAG is a framework to minimize hallucinations:

<img src="images/RAG.png"/>

## Let's get started!

### We'll be using:

<img src="images/ragna-logo.png" width=55%/>

RAG orchestration framework designed to scale from research to production.

<br>

<img src="images/panel-logo.png" width=50%/>

Powerful interactive dashboard and application development framework.

### 1. Provide relevant data

10-K reports from Ford, GM, and Tesla, as well as a file describing Ragna:

In [ ]:
documents = [
    "files/what-is-ragna.txt",
    "files/ford-10k-2022.pdf",
    "files/gm-10k-2022.pdf",
    "files/tesla-10k-2022.pdf",
]

open(documents[0], "r").read()

### 2. Preliminary setup and configuration

```bash
export OPENAI_KEY=XXX # Export relevant API keys

ragna init # Create ragna.toml config-file using CLI wizard
``` 

Using the configuration file, you can set the assistants, source storages, API endpoints, etc.

Create configuration using the file:

In [ ]:
from ragna import Config

config = Config.from_file('ragna.toml')
config

### 3. Select assistants & source storage:

In [ ]:
from ragna.assistants import Gpt4, Gpt35Turbo16k 
from ragna.source_storages import Chroma, LanceDB

from ragna.core import Rag

In [ ]:
rag = Rag(config)

### 4. Start a chat

In [ ]:
# Note: Embedding documents takes a few minutes

chat = rag.chat(documents=documents, 
                source_storage=Chroma,
                assistant=Gpt35Turbo16k,
               )

await chat.prepare() # Ragna is async by design

### 5. Ask your questions!

In [ ]:
answer = await chat.answer("What is Ragna?")
print(answer.content)

And, view sources:

In [ ]:
answer.sources[0].content

In [ ]:
print(answer)

Some more prompts :)

In [ ]:
answer = await chat.answer("Did Ford earn more than GM?")
print(answer.content)

In [ ]:
answer = await chat.answer("Did Tesla earn more than GM?")
print(answer.content)

In [ ]:
answer = await chat.answer("How much did Tesla, GM, and Ford earn")
print(answer.content)

### Aside: We can quickly compare assistants & source storages

In [ ]:
import asyncio
import itertools
from pprint import pprint

from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma, LanceDB

source_storages = [Chroma, LanceDB]
assistants = [Gpt35Turbo16k, Gpt4]

prompt = "How much did Tesla, GM, and Ford earn"

async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=sec_10k_documents,
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content

experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

In [ ]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

### 6. Create a Panel ChatFeed

TODO

In [ ]:
import panel as pn

pn.extension()

### 7. Connect it to Ragna

TODO

### 8/9/10. TBD / add steps as needed

## Ragna ships with a fully-featured UI built with Panel

This uses **Ragna's REST API**, which is more convenient to build production applications.

<img src="images/ragna-web-ui.gif" />

## Thank you! Question?

Learn more: [**ragna.chat**](https://ragna.chat/)

Please share your thoughts and feedback!